In [3]:
import numpy as np
import pandas as pd
from scipy import stats
from random import uniform,randint
import uuid

# Дельта метод

**Дельта метод** - это метод обсчета результатов АБ теста по исследованию метрик отношения. 

**Метрика отношения** - это когда у каждого пользователя в выборках есть множество каких-то значений и мы получаем новую метрику, усредняя их.
Например, у каждого юзера было множество покупок за период с разной ценой. Мы хотим узнать средний чек каждого юзера - это метрика отношения. 

**Суть метода:**
    
1. Каждому юзеру аггрегируем кол-во (Y) и сумму (X) его значений. Хотим исследовать метрику отношения X/Y.

2. Считаем дисперсии в каждой выборке по формуле:
    
$$
V\left(\frac{X}{Y}\right) \approx \frac{1}{|A|} \left( \frac{1}{\mu_Y^2} V(X) - \frac{2 \mu_X}{\mu_Y^3} \text{cov}(X, Y) + \frac{\mu_X^2}{\mu_Y^4} V(Y) \right)
$$

**X, Y** - столбцы с значениями числителя и знаменателя

**VX** - дисперсия значений числителей (напр. значений сумм покупок по юзерам).

**VY** - дисперсия значений знаменателей (напр. значений количеств покупок по юзерам)

|A| - размер выборки

mu_Х - среднее значений числителей 

mu_Y - среднее значений знаменателей



3. Считаем t-статистику по формуле:
$$
t = \frac{\mathcal{R}_B - \mathcal{R}_A}{\sqrt{V(\mathcal{R}_A) + V(\mathcal{R}_B)}} \sim N(0,1)
$$

**R** - полученная метрика отношения по всей выборке, то есть сумму всех числителей делим на всю сумму знаменателей (напр.: сумму всех чеков делим на сумму количеств всех покупок)

**VR** - дисперсия, вычисляемая по вышеуказанной формуле.


Минусы дельта-метода - нельзя применить CUPED и стартификацию.

## Пример (средний чек):

In [4]:
# получаем пример датафрейм с набором юзеров и их чеков:

# тестовая выборка:
users = [str(uuid.uuid4()) for i in range(100)]
bills = []

for user in users:
    bills_of_user = np.random.uniform(2, 11, randint(1,50)) # рандомное кол-во покупок 1..50 каждая ценой 2...11 (выше) 
    bills.append(bills_of_user)    

data_test = pd.DataFrame({'user_id':users, 'bill':bills}).explode('bill')

# контрольная выборка:
users = [str(uuid.uuid4()) for i in range(1000)]
bills = []

for user in users:
    bills_of_user = np.random.uniform(1, 10, randint(1,50)) # рандомное кол-во покупок 1..50 каждая ценой 1...10 
    bills.append(bills_of_user)    

data_control = pd.DataFrame({'user_id':users, 'bill':bills}).explode('bill')

data_control.head()

,user_id,bill
0,7b20ff20-d2a4-4986-a2e9-73983f978233,9.003749
0,7b20ff20-d2a4-4986-a2e9-73983f978233,9.102916
0,7b20ff20-d2a4-4986-a2e9-73983f978233,7.104547
0,7b20ff20-d2a4-4986-a2e9-73983f978233,4.439855
0,7b20ff20-d2a4-4986-a2e9-73983f978233,8.155702


In [5]:
# предподготавливаем данные:
preprocessed_test = data_test.groupby(['user_id'], as_index=False).agg({"bill":['sum', 'count']})
preprocessed_test.columns = ['user_id', 'bill_sum', 'bill_count']
preprocessed_test['bill_sum'] = preprocessed_test['bill_sum'].astype(float)

preprocessed_control = data_control.groupby(['user_id'], as_index=False).agg({"bill":['sum', 'count']})
preprocessed_control.columns = ['user_id', 'bill_sum', 'bill_count']
preprocessed_control['bill_sum'] = preprocessed_control['bill_sum'].astype(float)

print(f'Размер теста: {len(preprocessed_test)}')
print(f'Размер контроля: {len(preprocessed_control)}')

preprocessed_control.head()

Размер теста: 100
Размер контроля: 1000


,user_id,bill_sum,bill_count
0,000ecf9b-e50f-4580-83a8-4c7651acdac9,69.228778,11
1,002b29dc-ecef-4f27-af27-a5e1fb4aebc7,127.513361,27
2,004b8918-6e0d-41bb-995e-eb016d03355e,94.218713,16
3,005e5f4d-6935-4ad6-8987-2ce833944535,174.821271,33
4,010864b7-a254-4009-82e7-db7294f2f4e5,68.089787,13


In [8]:
# считаем дисперсии:

# тест:
avg_x = preprocessed_test['bill_sum'].mean()
avg_y = preprocessed_test['bill_count'].mean()
var_x = preprocessed_test['bill_sum'].var()
var_y = preprocessed_test['bill_count'].var()
n = len(preprocessed_test)
cov = np.cov(preprocessed_test['bill_sum'], preprocessed_test['bill_count'])[0, 1]

var_test = (var_x/(avg_y**2) - 2*avg_x*cov/(avg_y**3) + (avg_x**2)*var_y/(avg_y**4)) / n
# контроль:
avg_x = preprocessed_control['bill_sum'].mean()
avg_y = preprocessed_control['bill_count'].mean()
var_x = preprocessed_control['bill_sum'].var()
var_y = preprocessed_control['bill_count'].var()
n = len(preprocessed_control)
cov = np.cov(preprocessed_control['bill_sum'], preprocessed_control['bill_count'])[0, 1]

var_control = (var_x/(avg_y**2) - 2*avg_x*cov/(avg_y**3) + ((avg_x**2)*var_y)/(avg_y**4)) / n

print(f'Дисперсия теста: {var_test:.5f}')
print(f'Дисперсия контроля: {var_control:.5f}')

Дисперсия теста: 0.00296
Дисперсия контроля: 0.00026


In [9]:
# считаем t-статистику:
r_test = preprocessed_test['bill_sum'].sum() / preprocessed_test['bill_count'].sum()
r_control = preprocessed_control['bill_sum'].sum() / preprocessed_control['bill_count'].sum()

t_statistic = (r_test-r_control) / np.sqrt(var_test+var_control)

In [10]:
# считаем p-value:
pvalue = 2 * stats.norm.cdf(-abs(t_statistic))
pvalue

4.8021597489453304e-60

Формулу с MDE в этом случае надо использовать подставляя вместо станд. отклонения - корень из полученных дисперсий. 
MDE - мин. эффект в рассматриваемых единицах (напр. 10 руб.)

# Линеаризация:

**Линеаризация** - метод для обсчета результатов АБ теста на метрики отношения, который позволяет перейти к непрерывным пользовательнским метрикам, чтобы можно было применять CUPED, стартификацию и применять обычный T-тест.

**Суть метода:**
    
Берем числитель (**X**) и знаменатель (**Y**) и считаем коэф-т **k**, равный общей метрике отношения в контрольной выборке т.е. сумму всех X делить на сумму всех Y. И считаем линеаризованную метрику по формуле:
    
$$
L = X - kY 
$$

для каждого юзера

$$k = \frac{\sum X_i}{\sum Y_i}$$ (в контроле)

## Пример (средний чек):

In [20]:
# считаем коэф-т:
k = sum(preprocessed_control['bill_sum']) / sum(preprocessed_control['bill_count'])

# считаем линеаризированную метрику для каждой выборки:
preprocessed_control['lin_metric'] = preprocessed_control['bill_sum'] - k*preprocessed_control['bill_count']
preprocessed_test['lin_metric'] = preprocessed_test['bill_sum'] - k*preprocessed_test['bill_count']

# тут можно применить CUPED

# считаем pvalue:
pvalue = stats.ttest_ind(preprocessed_test['lin_metric'], preprocessed_control['lin_metric'])[1]
pvalue

2.525958461274059e-52